# Laboratorium Automatyka Pojazdowa: Klasyfikator znaków drogowych

## Część 1: Klasyfikacja

Problem do rozwiązania w ramach tego laboratorium jest zadaniem klasyfikacji: przypisania danym wejściowym - w tym wypadku zdjęciom znaków - etykiet - w tym wypadku: nazw typów znaków.

Do rozwiązania problemu należy przygotować konwolucyjną sieć neuronową, która będzie rozwiązywała to zagadnienie.

Podstawowa struktura konwolucyjnej sieci neuronowej to:
- grupa / grupy powtarzanych naprzemiennie warstw `Conv2D` oraz `MaxPooling2D`, które "tworzą" rosnącą (w wymiarze głębi) grupę tzw. map cech (feature maps; wizualizację tego procesu przedstawiono w [przejrzysty sposób w tym miejscu](https://what-when-how.com/wp-content/uploads/2012/07/tmp725d63_thumb.png))
- warstwa flatten (która "spłaszcza" wszystkie piksele z map cech do jednowymiarowego wektora)
- warstwa bądź warstwy gęsto połączone przetwarzające spłaszczone mapy cech
- warstwa wyjściowa będącą również warstwą gęsto połączoną, tyle że z funkcją aktywacji [softmax](https://en.wikipedia.org/wiki/Softmax_function) o liczbie neuronów równej liczbie klas - co sprawia, że każdy z neuronów w tej warstwie odpowiada jednej z klas i ma przyjmuje wartość będącą prawdopodobieństwem, że dane wejściowe odpowiadają danej klasie; f. aktywacji softmax w tej warstwie zapewnia spełnienie warunku, aby suma wspomnianych prawdopodobieństw była $\leqslant 1$

Jako funkcję straty optymalizatora należy wykorzystać (jest to już zaimplementowane) [SCCE (Sparse Categorical Cross-Entropy)](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy), czyli binarna entropia krzyżowa kodująca etykiety w postaci rosnącej liczby przyporządkowanej każdej z klas - sprawia to, że w ostatniej warstwie funkcją aktywacji będzie [sygmoida](https://en.wikipedia.org/wiki/Sigmoid_function).

Należy zatem:
- uzupełnić notatnik, aby był funkcjonalny, sieć trenowała się i realizowała opisane zadanie
- wykonać eksperymenty, zmieniając parametry zgodnie z tabelką na dole tego notatnika
- z każdego eksperymentu zanotować w notatniku wyniki (m. in. accuracy oraz ilość iteracji, po której nastąpiła zbieżność podczas trenowania) oraz zapisać obserwacje
- zapisywać do katalogu `img/` po wytrenowaniu i przetestowaniu każdej z finalnie dobranych struktur sieci neuronowej, wykresy:
    - struktury sieci - do pliku `modelX.png`: `model1.png`, `model2.png`, ...
    - krzywej skuteczności sieci i straty optymalizatora (jeden plik - są to subploty) w pliku o nazwie wskazanej w tabeli - do pliku `fittingX.png`: `fitting1.png`, `fitting2.png`, ...
    
    Zapisywanie wykresów z notatnika Jupyter jest bardzo proste, co zostało przedstawione poniżej:
    
    <img src="img/tutorial-saving-plot-to-file.png" width="80%">

    Należy pamiętać o zapisywaniu tych wykresów oraz poprawnym nazewnictwie - dzięki temu nie będzie konieczne dodatkowe wprowadzanie zmian w nazwach plików w tabelce z rezultatami na dole notatnika.

- zanotować w tablece na dole notatnika wnioski podsumowujące wpływ parametrów na zachowanie, skuteczność sieci i czas zbieżności jej trenowania



W tym notatniku Jupyter Notebook przygotowany został szablon, w którym miejsca oznaczone komentarzem `# TODO` należy zastąpić właściwym kodem. Niektóre fragmenty kody w okolicy takich komentarzy zastąpiono Ellipsis (`...`), które również należy usunąć przed uzupełnianiem.

### Zaimportowanie wykorzystywanych pakietów

In [ ]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from IPython.display import Image as JupyterImage
from typing import *
import pickle

### Sprawdzenie dostępności kart (oraz oprogramowania) GPU

In [ ]:
print("Available GPUs:")
for dev in tf.config.list_physical_devices('GPU'):
    details = tf.config.experimental.get_device_details(dev)

    devName = details.get('device_name', '?')
    computeCapabilityTup = details.get('compute_capability', '?')

    print(f"{dev.name} -> {devName}, compute capability {computeCapabilityTup[0]}.{computeCapabilityTup[1]}")
else:
    print("No GPUs detected")

### Pobranie oraz rozpakowanie zbioru danych


Do tego zadania wykorzystany został zbiór danych [chriskjm/polish-traffic-signs-dataset](https://www.kaggle.com/datasets/chriskjm/polish-traffic-signs-dataset), który zawiera w sobie:
- przycięte zdjęcia RGB polskich znaków drogowych o rozmiarach `256x256x3` ([`../data/classification/<nazwa klasy>/0...n.jpg`](../data/classification/)) do problemu klasyfikacji,
- nieprzycięte zdjęcia RGB polskich znaków drogowych do problemu lokalizacji:
    - ([`../data/detection/imgs/0...n.jpg`](../data/detection/imgs)) - zdjęcia nieprzyciętych znaków
    - ([`../data/detection/labels/0...n.txt`](../data/detection/labels)) - etykiety zdjęć: klasy wraz ze współrzędnymi bounding boxów (prostokątów opisanych na poszukiwanym obiekcie)

In [ ]:
if not os.path.exists("../data/polish-traffic-signs-dataset.zip"):
    !pip install kaggle
    !kaggle datasets download -d chriskjm/polish-traffic-signs-dataset
    !mv polish-traffic-signs-dataset.zip ../data/polish-traffic-signs-dataset.zip
else:
    print("Dataset zip present on disk")

if not os.path.exists("../data/classification") or not os.path.exists("../data/detection"):
    !unzip -q ../data/polish-traffic-signs-dataset -d ../data/
else:
    print("Extracted dataset present on disk")

### Załadowanie danych ze zbioru danych

Z pomocą funkcji `tf.keras.utils.image_dataset_from_directory` możliwe jest załadowanie danych do nowej instancji klasy `tf.data.Dataset`, która w efektywny sposób pozwala ładować dane w tzw. wsadach (batchach) za pomocą iterowalnego generatora, co pozwala na optymalizację użycia pamięci, gdyż cały dataset (który może być bardzo duży) nie jest ładowany z dysku, a w zamian pojedyczne jego partie są ładowane w chwili, gdy próbuje się uzyskać do nich dostęp.

Metoda przyjmuje dodatkowo parametry pozwalające skonfigurować sposób ładowania danych, m. in.:
- `labels` - `inferred` powoduje załadowanie etykiety danej próbki z nazwy zawierającego ją folderu
- `color_mode` - pozwala na np. binaryzację obrazów; w naszym przypadku będą one załadowane jako RGB - z 3 kanałami
- `batch_size` - kontroluje rozmiar wsadu - dataset można iterować po wsadach
- `image_size` - pozwala na prostą zmianę rozmiaru
- `shuffle` - losowo przetasowywuje kolejność próbek
- `validation_split` - pozwala na podział próbek na zbiór treningowy i walidacyjny (w tym przypadku jest on wykorzystywany jako testowy) - ten parametr oznacza znormalizowany % próbek, który trafi do zbioru walidacyjnego

In [ ]:
BATCH_SIZE = ... # TODO: no. of images in a single batch; start experimenting with 32
IMAGE_SIZE = ... # px, image size; set to 256

train_dataset: tf.data.Dataset
train_dataset: tf.data.Dataset
train_dataset, test_dataset = tf.keras.utils.image_dataset_from_directory(
    "../data/classification",
    labels='inferred',
    label_mode='int', # for SCCE loss
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    validation_split=..., # TODO: 80% train, 20% test - proszę sprawdzić w dokumentacji znaczenie parametru
    subset="both",
    seed=100
)

classNames = train_dataset.class_names
classesCount = len(classNames)
print(f"Loaded {classesCount} classes: {', '.join(classNames)}")

In [ ]:
with open("../models/classifierClassNames.pickle", "wb+") as f:
    pickle.dump(classNames, f)

Poniższa komórka wyświetla 25 losowych znaków z treningowego zbioru danych wraz etykietami:

In [ ]:
PREVIEW_COLS = 5
PREVIEW_ROWS = 5
FIG_UNIT_SIZE = 2

fig, axs = plt.subplots(nrows=PREVIEW_ROWS, ncols=PREVIEW_COLS, figsize=(PREVIEW_COLS * FIG_UNIT_SIZE, PREVIEW_ROWS * FIG_UNIT_SIZE))
print(f"Displaying {PREVIEW_COLS * PREVIEW_ROWS} random train dataset samples")
fig.tight_layout()

# create an iterator to a copy of the dataset unbatched, i.e., iterated not in batches of shape (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), but (IMAGE_SIZE, IMAGE_SIZE, 3) - single RGB 256x256 images
samplesIter = iter(train_dataset.unbatch())

for i in range(PREVIEW_ROWS):
    for j in range(PREVIEW_COLS):
        sample = next(samplesIter)

        image, classIndex = sample
        classIndex = classIndex.numpy() # tf.Tensor -> np.ndarray
        classLabel = classNames[classIndex]
        
        axs[j][i].imshow(image.numpy().astype(np.uint8)) # tf.Tensor -> np.ndarray, np.ndarray as uint8 ([0, 255])
        axs[j][i].set_title(classLabel)

### Stworzenie struktury modelu

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="input"),
    # TODO: proszę dodać dwukrotnie:
    # - najpierw warstwę konwolucyjną o o 30 filtrach i rozmiarze kernela 3x3 oraz f. aktywacji relu - https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
    # - następnie warstwę max pooling 2D o pool size 2x2 - https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
    tf.keras.layers.Flatten(),
    # TODO: proszę dodać warstwę Dense o 512 neuronach i funkcji aktywacji relu - https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
    tf.keras.layers.Dense(classesCount, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.001
    ),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)
model.summary()

tf.keras.utils.plot_model(model, to_file="model.png", show_shapes=True, show_layer_activations=True, show_trainable=True, show_dtype=True, show_layer_names=True)
JupyterImage(filename='model.png', width="500px")

### Trenowanie modelu

In [ ]:
EPOCHS = 30
history = model.fit(train_dataset, epochs=EPOCHS, validation_data=test_dataset, validation_freq=1)

In [ ]:
history_dict = history.history

fig, axes = plt.subplots(1, 2, figsize=(16, 5))
axes = axes.ravel()
keys = list(history_dict.keys())
for k in keys:
  ax = axes[0 if k.endswith("loss") else 1]

  ax.plot(history_dict[k], label=k)
  ax.grid()

for ax in axes:
  ax.legend()

axes[0].set_title("Optimizer loss")
axes[1].set_title("Model accuracy")

test_loss, test_acc = model.evaluate(test_dataset)
print(f"Model test loss: {test_loss :.2f}, test accuracy: {test_acc * 100 :.2f}%")

### Klasyfikacja losowych próbek - wizualizacja

In [ ]:
PREVIEW_COLS = 5
PREVIEW_ROWS = 5
FIG_UNIT_SIZE_W, FIG_UNIT_SIZE_H = 4, 4.5

fig, axs = plt.subplots(nrows=PREVIEW_ROWS, ncols=PREVIEW_COLS, figsize=(PREVIEW_ROWS * FIG_UNIT_SIZE_H, PREVIEW_COLS * FIG_UNIT_SIZE_W))
print(f"Classifying {PREVIEW_COLS * PREVIEW_ROWS} random train dataset samples")
fig.tight_layout()

# create an iterator to a copy of the dataset unbatched, i.e., iterated not in batches of shape (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), but (IMAGE_SIZE, IMAGE_SIZE, 3) - single RGB 256x256 images
samplesIter = iter(train_dataset.unbatch())

for i in range(PREVIEW_ROWS):
    for j in range(PREVIEW_COLS):
        sample = next(samplesIter)

        image, classIndex = sample
        classIndex = classIndex.numpy() # tf.Tensor -> np.ndarray
        classLabel = classNames[classIndex]

        predictions = model.predict(image.numpy().reshape(1, IMAGE_SIZE, IMAGE_SIZE, 3))[0]
        classPredictionIndex = np.argmax(predictions)
        classPredictionProbability = predictions[classPredictionIndex]
        classPredictionLabel = classNames[classPredictionIndex]
        
        axs[j][i].imshow(image.numpy().astype(np.uint8)) # tf.Tensor -> np.ndarray, np.ndarray as uint8 ([0, 255])
        axs[j][i].set_title(f"Pred: {classPredictionLabel}, real: {classLabel}, prob: {classPredictionProbability * 100 :.1f}% ({'OK' if classPredictionLabel == classLabel else 'FAILURE'})")
        axs[j][i].axis("off")

In [ ]:
# wyeksportowanie grafu inferencji - zoptymalizowanego formatu modelu wraz z wyuczonymi wagami, na którym można efektywnie przeprowadzać inferencję - tj. predykcję dla dowolnych danych wejściowych

model.save('../models/classifier.keras')

### Wyniki, wnioski i obserwacje


|             Architektura sieci              | Czas zbieżności <br/> uczenia [epochs] | Skuteczność na zbiorze<br/>testowym po osiągnięciu<br/>zbieżności (accuracy) [%] | Czas do wystąpienia<br/>overfittingu [epochs] |           Wykres krzywych uczenia             |
| :-----------------------------------------: | :------------------------------------: | :------------------------------------------------------------------------------: | :-------------------------------------------: | :------------------------------------------:  |
| <img src="./img/model1.png?a=1" width="400px">  |                    X                   |                                         X                                        |                     X                         | <img src="./img/fitting1.png?a=1" width="700px"> |
| <img src="./img/model2.png" width="400px">  |                    X                   |                                         X                                        |                     X                         | <img src="./img/fitting2.png" width="700px"> |
| <img src="./img/model3.png" width="400px">  |                    X                   |                                         X                                        |                     X                         | <img src="./img/fitting3.png" width="700px"> |
| <img src="./img/model4.png" width="400px">  |                    X                   |                                         X                                        |                     X                         | <img src="./img/fitting4.png" width="700px"> |
| <img src="./img/model5.png" width="400px">  |                    X                   |                                         X                                        |                     X                         | <img src="./img/fitting5.png" width="700px"> |
| <img src="./img/model6.png" width="400px">  |                    X                   |                                         X                                        |                     X                         | <img src="./img/fitting6.png" width="700px"> |

Wnioski:
...